# Multiple Agent Chat

### Requirement

Auto Agent setup need `vllm>=0.2.0`, you can use the following command to install the latest version of [vLLM](https://github.com/vllm-project/vllm).
```
pip install vllm
```

In [ ]:
!pip install vllm

In [ ]:
import autogen
import subprocess as sp
from typing import *

In [ ]:
class AgentCreator:
    
    agent_procs: Dict[str, sp.Popen] = {}
    agent_procs_assign: Dict[str, Tuple[autogen.AssistantAgent, str]] = {}
    openai_server_name: str = 'openai'
    
    def __init__(
        self,
        logging_path: Optional[str] = "logs/",
    ):
        self.logging_path = logging_path
        
    def get_config(
        self,
        model_name_or_hf_repo: str,
        world_size: Optional[int] = 1,
        host: Optional[str] = "127.0.0.1",
        port: Optional[str] = "8000",
        seed: Optional[int] = 42,
        temperature: Optional[int] = 0,
    ) -> Tuple[List, str]:
    
        if "gpt-" in model_name_or_hf_repo:
            config = autogen.config_list_from_json(
                "OAI_CONFIG_LIST",
                filter_dict={
                    "model": [model_name_or_hf_repo]
                },
            )
            server_id = self.openai_server_name
        else:
            model_name = model_name_or_hf_repo.split('/')[-1]
            config = [autogen.get_config_list(
                api_keys = ["EMPTY"],
                api_bases = [f"{host}:{port}"],
            )]
            server_id = f'{model_name}_{host}_{port}_{seed}_{temperature}'
            if self.agent_procs.get(server_id, None) is None:
                # Use vllm to set up a server with OpenAI API support
                agent_proc = sp.Popen(['python', '-m', 'vllm.entrypoints.openai.api_server',
                                       '--host', host,
                                       '--port', port,
                                       '--model', model_name_or_hf_repo,
                                       '--tensor-parallel-size', world_size,
                                       '>', f"{self.logging_path}/serving_{model_name}.log"])
                self.agent_procs[server_id] = agent_proc
        
        return config, server_id
    
    def create_agent(
        self, 
        agent_name: str,
        seed: int = 42,
        temperature: int = 0,
        **kwargs
    ) -> autogen.AssistantAgent:
        config, server_id = self.get_config(seed=seed, temperature=temperature, **kwargs)
        agent = autogen.AssistantAgent(name=agent_name,
                                       llm_config={
                                           "seed": seed,  # seed for caching and reproducibility
                                           "config_list": config,  # a list of OpenAI API configurations
                                           "temperature": temperature,  # temperature for sampling
                                       }, **kwargs)
        self.agent_procs_assign[agent_name] = (agent, server_id)
        return agent

    def clear_agent(
        self,
        agent_name: str,
        recycle_endpoint: bool = True
    ):
        _, server_id = self.agent_procs_assign[agent_name]
        del self.agent_procs_assign[agent_name]
        if recycle_endpoint:
            if server_id == self.openai_server_name:
                return
            else:
                for _, iter_sid in self.agent_procs_assign.values():
                    if server_id == iter_sid:
                        return
                self.agent_procs[server_id].terminate()
    
    def start(self,
              task: str,
              manager_config: dict = {},
              max_round: Optional[int] = 12,
              init_messages: Optional[List[dict]] = [],
              user_name: Optional[str] = "User Proxy",
              user_system_message: Optional[str] = "A human admin.",
              **kwargs):
        agent_list = [agent for agent, _ in self.agent_procs_assign]
        user_proxy = autogen.UserProxyAgent(
            name=user_name,
            system_message=user_system_message,
            **kwargs
        )
        agent_list.append(user_proxy)
        groupchat = autogen.GroupChat(agents=agent_list, messages=init_messages, max_round=max_round)
        manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=manager_config)
        
        user_proxy.initiate_chat(manager, message=task)
        